In [16]:
import pandas as pd
import geopandas as gpd

data = pd.read_csv("GVA_w-injury_2015_2021apr_geocoded.csv")

data = data.rename(columns={"X": "longitude", "Y": "latitude", "city_or_county": "city"})
data["latitude"] = pd.to_numeric(data["latitude"], errors='coerce')
data["longitude"] = pd.to_numeric(data["longitude"], errors='coerce')
data['n_killed'] = data['n_killed'].fillna(0)
data['n_injured'] = data['n_injured'].fillna(0)

data = data.drop(columns=['incident_url', 'source_url'])

data.to_csv("GunViolence_cleaned.csv", index=False)

gdf = gpd.GeoDataFrame(
    data,
    geometry=gpd.points_from_xy(data.longitude, data.latitude),
    crs="EPSG:4326"
)

gdf[["date","state","city","address","n_killed","n_injured","month","year","longitude","latitude"]].to_csv(
    'GunViolenceArchive_clean.csv', index=False
)